# Intro to neural networks and back-propagation

[Andrej Karpathy](https://karpathy.ai/)

[YouTube video link](https://youtu.be/VMj-3S1tku0?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ)

> This is the most step-by-step spelled-out explanation of backpropagation and training of neural networks. It only assumes basic knowledge of Python and a vague recollection of calculus from high school.

https://github.com/karpathy/micrograd

In [ ]:
# Run ONCE to update any new kernel instance.
# You MUST restart the kernel after updating.
!pip install --upgrade pip
!pip install graphviz
!apt-get update
!apt-get install -y graphviz
!pip install torch
print('Complete!')

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Derivative of simple function

$L = \lim\limits_{x \to \infty} \frac{f(x+h)-f(x)}{h}$

In [ ]:
def f(x):
    return 3*x**2 - 4*x + 5 # derivative is 6x-4

In [ ]:
f(3.0)

In [ ]:
xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs, ys)

In [ ]:
h = 0.000001
x = 3.0
(f(x + h) - f(x))/h # derivative

In [ ]:
# let's get more complex
a = 2.0
b = -3.0
c = 10.0
d = a*b + c
print(d)

In [ ]:
h = 0.0001

# inputs
a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
a += h
d2 = a*b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2-d1)/h) # derivative

## Value Object

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0 # the gradient for back-propagation
        self._backward = lambda: None
        self._prev = set(_children) #convert for efficiency
        self._op = _op
        self.label = label
        
    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad # Multivariate chain rule for when self=other
            other.grad += 1.0 * out.grad  
        out._backward = _backward
        
        return out
    
    def __radd__(self, other): # other + self
        return self + other
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad  += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other): # other * self
        return self * other
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * self.data**(other-1) * out.grad
        out._backward = _backward
        
        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
    
    def backward(self):
        # We need to call _backward() right-to-left (output-to-input) so we need to topologically sort the network
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
    
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a * b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L

### Visualize Graph

In [ ]:
from graphviz import Digraph

def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name=uid, label="{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            # if this value is the result of some operation, create an op node for it
            dot.node(name = uid + n._op, label=n._op)
            # and connect this node to it
            dot.edge(uid + n._op, uid)
            
    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
    return dot

In [ ]:
draw_dot(L)

## Back-Propagation

We calculate the derivative of a node's weight with-respect-to 'L' (loss-function). This is called the _gradient_ and tells us how much a change in the node's weight impacts the loss-function 'L'.

Chain Rule: $\frac{dz}{dx}=\frac{dz}{dy}*\frac{dy}{dx}$

> "If a car travels twice as fast as a bicycle and the bicycle is four times as fast as a walking man, then the car travels 2 x 4 = 8 times as fast as the man." - George F. Simmons

### Propagation
$\frac{dL}{dL}=1.0$

$L=d*f$
- $\frac{dL}{dd}=f$
- $\frac{dL}{df}=d$

$d=c+e$
- $\frac{dd}{dc}=1.0$ (local derivative)
- $\frac{dd}{de}=1.0$
- $\frac{dL}{dc} \implies \frac{dL}{dd}*\frac{dd}{dc} \implies f*1.0=f$ (because the local derivative is 1, addition implies a pass-through of the gradient from the previous node)
- $\frac{dL}{de} \implies \frac{dL}{dd}*\frac{dd}{de} \implies f*1.0=f$

$e=a*b$
- $\frac{de}{da}=b$ (local derivative)
- $\frac{de}{db}=a$
- $\frac{dL}{da} \implies \frac{dL}{de}*\frac{de}{da} \implies f*b$
- $\frac{dL}{db} \implies \frac{dL}{de}*\frac{de}{db} \implies f*a$

In [ ]:
# Manual back propagation
L.grad = 1.0
f.grad = d.data
d.grad = f.data
c.grad = d.grad
e.grad = d.grad
a.grad = e.grad * b.data
b.grad = e.grad * a.data

In [ ]:
draw_dot(L)

In [ ]:
def lol():
    # create a little isolated playground to verify the derivates/gradients
    h = 0.0001
    
    a = Value(2.0, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0, label='f')
    L = d * f; L.label = 'L'
    L1 = L.data
    
    a = Value(2.0, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    a.data += h # nudge 'a' by 'h'
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0, label='f')
    L = d * f; L.label = 'L'
    L2 = L.data
    
    print((L2 - L1)/h)
    
lol()

## Neuron Model

For a given set of inputs $\{x_1, x_2, \ldots\}$ and corresponding weights $\{w_1, w_2, \ldots\}$, a neuron is defined as:

$f(\displaystyle\sum_{i} w_i x_i  + b)$

where:
- $b$ is the neuron bias
- $f()$ is the activation function, such as $\tanh$

In [ ]:
# Plot tanh() function
plt.plot(np.arange(-5,5,0.2), np.tanh(np.arange(-5,5,0.2))); plt.grid()

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813715870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'

In [ ]:
draw_dot(o)

In [ ]:
# manual back propagation
o.grad = 1.0
# o = tanh(n) => do/dn = 1 - tanh(n)**2 => 1 - o**2
n.grad = 1 - o.data**2
b.grad = n.grad
x1w1x2w2.grad = n.grad
x1w1.grad = x1w1x2w2.grad
x2w2.grad = x1w1x2w2.grad
x1.grad = x1w1.grad * w1.data
w1.grad = x1w1.grad * x1.data
x2.grad = x2w2.grad * w2.data
w2.grad = x2w2.grad * x2.data

In [ ]:
# Init auto-back-propagation
o.grad = 1.0
o._backward()
n._backward()
b._backward()
x1w1x2w2._backward()
x1w1._backward()
x2w2._backward()

In [ ]:
# Once we define the backward() function in the Value class, we can simply call it on the output node.
o.backward()

In [ ]:
# We have a bug...
a = Value(3.0, label='a')
b = a + a; b.label = 'b' # Reusing 'a' causes a problem until we fix Value
b.backward()
draw_dot(b)

In [ ]:
# Another example of the bug...
a = Value(-2.0, label='a')
b = Value(3.0, label='b')
d = a * b; d.label = 'd'
e = a + b; e.label = 'e'
f = d * e; f.label = 'f'

f.backward()
draw_dot(f)

### Implementing tanh as exp

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813715870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
# ---
#o = n.tanh(); o.label = 'o'
# ---
e = (2*n).exp()
o = (e - 1) / (e + 1) # alternate tanh definition
o.label = 'o'

o.backward()
draw_dot(o)

## PyTorch

In [ ]:
import torch

# by default, tensor.dtype = float32
x1 = torch.Tensor([2.0]).double();               x1.requires_grad = True # pythorch assumes no gradients (for efficiency)
x2 = torch.Tensor([0.0]).double();               x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double();              w1.requires_grad = True
w2 = torch.Tensor([1.0]).double();               w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double(); b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())
print('---')

# pytorch keeps track of the gradient function for back propagation
print(o)

## Multilayer Perceptron

In [ ]:
import random

class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b) # activation function
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]

# A layer is just a list of neurons
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
# Multilayer perceptron: takes a list or 'nouts' to define each internal layer
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts  # list of layer counts with 'nin' as the first element
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


In [ ]:
# MLP: 3 input values, 2 inner layers of 4, and 1 output value
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

#draw_dot(n(x))

In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [ ]:
for k in range(20):
    # forward pass
    ypred = [n(x) for x in xs]

    # loss function (mean-squared error)
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
    
    # backward pass (gradient decent)
    for p in n.parameters():
        p.grad = 0.0 # DON'T FORGET TO DO THIS!
    loss.backward()
    
    #update
    for p in n.parameters():
        p.data += -0.1 * p.grad # negative sign to reduce the loss; not maximize it
        
    print(k, loss.data)

In [ ]:
# see the predictions now
ypred